In [233]:
from datetime import datetime

def formatDOB(datetime_string: str) -> str:

    # Define the format for parsing
    format_string = "%Y-%m-%dT%H:%M:%S.%fZ"

    parsed_datetime = datetime.strptime(datetime_string, format_string)

    dob_string = str(parsed_datetime.year) + "-" + str(parsed_datetime.month) + "-" + str(parsed_datetime.day)

    return dob_string

In [242]:
import requests
import json
from typing import Callable

def getTestUserValues(api_endpoint: str, data_access: Callable) -> dict:
    response = requests.get(api_endpoint)

    users = []
    response = response.text
    

    # Parse the JSON string into a Python dictionary
    data = json.loads(response)

    # Access specific key-values
    first_name, last_name, email, user_name, password, dob = data_access(data)

    return {"first_name": first_name, "last_name": last_name, "email": email, 
    "user_name": user_name, "password": password, "dob": dob}

In [243]:
import psycopg2

def insertIntoDB(users: dict):
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="testdata",
            user="camerondudley",
            password="123"
        )

        # Create a cursor object
        cur = conn.cursor()

        # Now you can execute SQL queries using the cursor
        first_name = users["first_name"]
        last_name = users["last_name"]
        user_name = users["user_name"]
        email = users["email"]
        password = users["password"]
        dob = users["dob"]
        sql = "INSERT INTO Users (first_name, last_name, user_name, email, password, date_of_birth, description) VALUES (%s, %s, %s, %s, %s, %s, '')"
        data = (first_name, last_name, user_name, email, password, dob)
        cur.execute(sql, data)

        # Commit changes to database now
        conn.commit()

    except psycopg2.Error as e:
        print(e, e.args)
    finally:
        # Close the cursor and the connection when done
        cur.close()
        conn.close()

In [ ]:
api_endpoint = 'https://randomuser.me/api/'

def getData(data_access: dict):
    first_name = data_access["results"][0]["name"]["first"]
    last_name = data_access["results"][0]["name"]["last"]
    email = data_access["results"][0]["email"]
    user_name = data_access["results"][0]["login"]["username"]
    password = data_access["results"][0]["login"]["password"]
    dob = formatDOB(data_access["results"][0]["dob"]["date"])
    return first_name, last_name, email, user_name, password, dob


try:
    for i in range(0, 500000):
        # first_name, last_name, email, user_name, password, dob
        user = getTestUserValues(api_endpoint, getData1)

        #put into db
        insertIntoDB(user)

except requests.exceptions.RequestException as e:
    print("An error occurred: {e}")

In [ ]:
import random

def getData(data_access: dict):
    data_access = res["data"][0]
    first_name = data_access["firstname"]
    last_name = data_access["lastname"]
    email = data_access["email"]
    user_name = data_access["username"]
    password = data_access["password"]
    month = random.randint(1, 12)
    day = random.randint(1, 25)
    year = random.randint(1975, 2005)
    dob = str(year) + "-" + str(month) + "-" + str(day)
    return first_name, last_name, email, user_name, password, dob


try:
    for i in range(0, 500000):
        api_endpoint = "https://fakerapi.it/api/v1/users?_quantity=1&_gender="
        num = random.randint(0,1)
        if num == 0:
            api_endpoint = api_endpoint + "male"
        elif num == 1:
            api_endpoint = api_endpoint + "female"
        # first_name, last_name, email, user_name, password, dob
        user = getTestUserValues(api_endpoint, getData)

        #put into db
        insertIntoDB(user)

except requests.exceptions.RequestException as e:
    print("An error occurred: {e}")